<a href="https://colab.research.google.com/github/zilavalencia/ChungaraVZila-IA-SIS420/blob/main/Segundo%20Parcial/aprendizaje_por_refuerzo_(4_en_raya).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/axr/blob/master/axr/00_intro.ipynb)

# Pregunta 2. Aprendizaje por refuerzo aplicado a 4 en raya

INTERACCIÓN: Hay un mundo o un entorno donde los algoritmos de axr (agentes) viven. éstos agentes interactúan con éste mundo de forma iterativa llevando a cabo diferentes acciones y recibiendo recompensas.\
si las acciones son correctas -> recompensa positiva.\
cuando reciben re.positivas los agentes refuerzan esas acciones buenas y cuando reciben re.negativas a no hacerlas tanto.


## BALANCE ENTRE EXPLORACIÓN Y EXPLOTACIÓN (PRINCIPAL PROBLEMA):
cuando tenemos agentes interactuando con un mundo llevando a cabo acciones, qué queremos?\
1. que el agente pruebe todas las acciones posibles a ver que pasa ó\
2. que una vez encuentre 1,2,3 acciones buenas símplemente haga esas y ya está. pero\
3. si de forma aleatoria explora otras acciones quizas encuentre algunas que todavía son mejores. (no habría encontrado si se enfoca sólo en las acciones buenas que encontró previamente).\
éste es un problema que aún no está resuelto en el axr.\
-pregunta que tenemos que hacernos: ¿qué queremos? ¿agentes que exploren más, agenten que exploten más?


VAMOS HACER EL TEMA DE LOS JUEGOS, VAMOS HACER AGENTES QUE DE FORMA AUTÓNOMA APRENDAR A JUGAR JUEGOS, IDEALMENTE MEJOR QUE UNA PERSONA.

## Elementos del Aprendizaje por refuerzo
-Politica: Comportamiento del agente en cada momento\
-Recompensa: recompensa numérica\
-Función de valor: El valor de la recompensa a largo plazo\
-Modelo del entorno: Comportamiento del entorno, sirve para planificar acciones.


POLÍTICA: Define como se comporta nuestro agente, qué acciones va a tomar en función del estado de éste mundo en el que vive. esta política puede tener muchas formas: lo pongo aquí en una tabla, puede ser un proceso de búsqueda, puede ser una red neuronal o símplemente llevar a cabo acciones aleatorias. son diferentes políticas que en ciertos casos van a funcionar mejor y en otros casos peor. un poco el objetivo de nuestro agente es que aprenda una política óptima.\
RECOMPENSA: Es un valor numérico que el agente recibe del mundo en el que vive cuando lleva a cabo acciones. lo utilizará como objetivo de aprendizaje, queremos que éstos agentes maximicen ésta recompensa. la política que maximize ésta recompensa será la política óptima.\
FUNCIÓN DE VALOR: Es una función que nos va a decir para cada estado en el que el agente se encuentre, o cada par de estado-acción -> le vamos asignar un valor que nos va a decir lo bueno que es: si nuestro agente se encuentra en el 'estado s' y desde el 'estado s' puede tomar 3 acciones (a1, a2 o a3). cada una de éstas acciones te llevará a un nuevo estado. si sabemos el valor de ese nuevo estado, podemos coger aquella acción que nos dé un valor más grande.

$$
    Q_{n+1} = Q_n + \alpha [R_n - Q_n]
$$
Qn+1 (NUEVO VALOR DE Q) -> EL VALOR DE Q SE VA IR ACTUALIZANDO DE MANERA ITERATIVA.\
Qn -> VALOR VIEJO\
alpha CONSTANTE ENTRE 0 Y 1. ES EL RATE DE APRENDIZAJE, LEARNING RATE.\
Rn - Qn  -> DIFERENCIA ENTRE LA RECOMPENSA OBTENIDA Y EL VALOR DE Q QUE TENEMOS HASTA ESE MOMENTO.

MODELO DEL ENTORNO: Nuestros agentes viven en un mundo que puede ser el mundo real o un mundo virtual. necesitamos modelar de alguna forma éste entorno asi sabremos qué recompensa le tenemos que dar al agente, si es positiva o negativa.



## Aprendizaje por refuerzo aplicado a un juego 4 en Raya


\EXPLICACIÓN DEL ALGORITMO QUE VAMOS A UTILIZAR:

•	Tenemos 2 agentes que van a jugar entre sí al tres en raya muchas veces (muchas iteraciones).\
•	Se van a ir guardando todos los estados en los que se encuentren (configuraciones de la tabla tres en raya-> cada uno de los estados) en una lista, en una tabla.\
•	A  cada uno de éstos estados le va aplicar un valor. en éste caso nuestra función de valor es la tabla de cuatro en raya que además utilizaremos como política.\
•	La idea es que en cada turno nuestro agente sabrá todas las posibles acciones que puede llevar a cabo-> todos los sitios en los que puede poner una 'x' Ó una 'o' en función del jugador que sea.\
•	Y como sabemos el valor de cada uno de éstos estados, podemos coger siempre aquella acción que tenga un valor más alto porque en promedio será el que nos de mayor recompensa el que nos haga ganar más veces al final la partida.




In [1]:
import numpy as np


class Board():                                                                  #CLASE BOARD(), REPRESENTA LA CUADRÍCULA
    def __init__(self):
        self.state = np.zeros((4,4))              #INICIALIZAMOS UNA MATRIZ DE 3*3 CON CEROS QUE REPRESENTA LAS DIFERENTES POSICIONES.
                                                  #EL ESTADO LO REPRESENTA COMO UN TABLERO.
    def valid_moves(self):                        #ÉSTA FUNCIÓN DEVUELVE TODOS LOS POSIBLES MOVIMIENTOS QUE SON VÁLIDOS-> TODAS LAS CASILLAS QUE ESTÁN LIBRES EN CADA MOMENTO
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0]

    def update(self, symbol, row, col):           #FUNCIÓN UPDATE -> VA ASIGNAR LA 'X' ó LA 'O', EN ÉSTE CASO SERÁN '1s' ó '-1s' A LA FILA Y COLUMNA EN CONCRETO
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError ("movimiento ilegal !")

    def is_game_over(self):                       #FUNCIÓN PARA COMPROBAR SI YA HEMOS TERMINADO EL JUEGO
        # comprobar filas y columnas
        if (self.state.sum(axis=0) == 4).sum() >= 1 or (self.state.sum(axis=1) == 4).sum() >= 1:    #COMPRUEBA SI TENEMOS 3 unos EN HORIZONTAL O VERTICAL PARA TODAS LAS FILAS Y COLUMNAS
            return 1
        if (self.state.sum(axis=0) == -4).sum() >= 1 or (self.state.sum(axis=1) == -4).sum() >= 1:  #AQUÍ COMPRUEBA PARA 'O' QUE ES '-1s'
            return -1
        # comprobar diagonales                                                                      #COMPRUEBA EN DIAGONAL
        diag_sums = [
            sum([self.state[i, i] for i in range(4)]),
            sum([self.state[i, 4 - i - 1] for i in range(4)]),
        ]
        if diag_sums[0] == 4 or diag_sums[1] == 4:
            return 1
        if diag_sums[0] == -4 or diag_sums[1] == -4:
            return -1
        # empate                                    #SI LA COLUMNA ESTÁ LLENA PERO NO SE CUMPLEN NINGUNA DE LAS CONDICIONES ANTERIORES ES UN EMPATE
        if len(self.valid_moves()) == 0:
            return 0
        # seguir jugando                            #Y SINO SE SEGUIREMOS JUGANDO
        return None

    def reset(self):                                #FUNCIÓN reset() VUELVE A PONER A CERO EL ESTADO PARA LA SIGUIENTE ITERACIÓN
        self.state = np.zeros((4,4))


In [ ]:
from tqdm import tqdm

class Game():                                                                   #CLASE GAME:SE ENCARGA DE LLEVAR A CABO TODAS LAS PARTIDAS
    def __init__(self, player1, player2):                                       #LE VAMOS A PASAR LOS DOS JUGADORES: player1 y player2
        player1.symbol = 1                                     #AL player1 LE ASIGNAMOS EL SÍMBOLO 1
        player2.symbol = -1                                    #AL player2 LE ASIGNAMOS EL SÍMBOLO -1
        self.players = [player1, player2]                      #TENEMOS " JUGADORES"
        self.board = Board()    #instancio un tablero, el objeto que he construido en la parte anterior


    def selfplay(self, rounds=100):                            #FUNCIÓN selfplay(), LE PASAMOS UN NÚMERO DE RONDAS, POR DEFECTO 100
                                                               #VA A COGER A LOS DOS PLAYERS (player1 y player2) QUE VAN A SER DOS AGENTES
                                                               #Y VAN A JUGAR EL UNO CONTRA EL OTRO TODAS LAS RONDAS. PARA CADA RONDA HASTA
                                                               #QUE LA PARTIDA ACABE-> QUE GANE EL JUGADOR1, QUE GANE EL JUGADOR2, O QUE HAYA UN EMPATE
        wins = [0, 0]
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()                                  #resetea el tablero al empezar el juego
            for player in self.players:
                player.reset()
            game_over = False                                   #pone en falso

            while not game_over:                                #ÉSTE BUCLE SE VA ESTAR EJECUTANDO SIEMPRE QUE LA VARIABLE game_over ESTÉ A False
                for player in self.players:                     #Y TENEMOS POR LOS DIFERENTES JUGADORES QUE SON SÓLO DOS...
                    action = player.move(self.board)            #...COGIENDO UNA ACCIÓN. CADA player LLAMARÁ A SU FUNCIÓN move() Y NOS DARÁ UNA action.
                    self.board.update(player.symbol, action[0], action[1])      #Hay dos acciones. VAMOS HACER EL UPDATE() DE LA TABLA

                    for player in self.players:
                        player.update(self.board)               #Y VAMOS IR GUARDANDO ÉSTE ESTADO. ÉSTE update() HACE QUE ÉSTE JUGADOR SE GUARDE EL ESTADO DE LA TABLA
                    if self.board.is_game_over() is not None:
                        game_over = True
                        break
                                                                #LOS DOS AGENTES SE IRÁN GUARDANDO TODOS ESTOS ESTADOS (while->break) HASTA QUE EL JUEGO ACABE
            self.reward()                                       #SALIMOS DEL BUCLE Y AQUÍ LE DAMOS LA RECOMPENSA

            #ADEMÁS VOY A ESTAR GUARDÁNDOLAS PARA CADA PARTIDA QUIEN GANA PARA LUEGO COMPARAR ÉSTOS AGENTES:
            for ix, player in enumerate(self.players):          #saca el índice 0 o 1   ix
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1                               #Va a aumentar sus victorias
        return wins


    def reward(self):                                           #FUNCIÓN reward() ASIGNA LA RECOMPENSA
        winner = self.board.is_game_over()                      #VEMOS QUIEN ES EL GANADOR (LA FUNCIÓN is_game_over() RETORNA 1,-1 ó 0)
        if winner == 0: # empate                                #SI HAY UN EMPATE LE DAMOS UNA RECOMPENSA DE...
            for player in self.players:
                player.reward(0.5)                              #...0.5 A AMBOS JUGADORES.

        else: # le damos 1 recompensa al jugador que gana       #SI NO HAY EMPATE HA GANADO EL UNO O EL OTRO
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)                            #AL QUE HA GANADO LE DAMOS UNA RECOMPENSA DE 1
                else:
                    player.reward(0)                            #AL QUE HA PERDIDO LE DAMOS UNA RECOMPENSA DE 0

In [ ]:
class Agent():                                                  #CLASE AGENTE(). VAMOS A VER CÓMO VA A APRENDER A JUGAR ÉSTE AGENTE DURANTE LAS PARTIDAS,
                                                                #DE CERO NO VA A SABER NADA.
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {} # tabla con pares estado -> valor              #NUESTRO AGENTE VA A TENER ÉSTA FUNCIÓN DE VALOR, UN dic {} QUE EN PYTHON SE INTERPRETA COMO UNA TABLA
                                                                                #TABLA {}: EN UNA COLUMNA VAMOS A GUARDAR TODOS LOS ESTADOS QUE VEA EL AGENTE
                                                                                #DURANTE TODAS LAS PARTIDAS Y CADA UNO DE ELLOS LE VA A ASIGNAR UN VALOR
        self.alpha = alpha         # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida.       #COMO SE LLEGA DE UN LUGAR A OTRO
        self.prob_exp = prob_exp   # probabilidad de explorar                   #AQUÍ TAMBIÉN TIENE UNA PROBABILIDAD DE EXPLORAR. LUEGO VEREMOS QUE NUESTRO AGENTE
                                                                                #VA A PROBAR ACCIONES ALEATORIAS, EN ESTE CASO CON UNA PROBABILIDAD DE 50%
                                                                                #VER ARRIBA: prob_exp=0.5-> ÉSTO ES ALGO QUE PODEMOS CONTROLAR
                                                                                #Y CON UNA PROB. DE 50% VA A COGER LA MEJOR ACCIÓN QUE HAYA ENCONTRADO HASTA
                                                                                #ESE MOMENTO...

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):                                        #...ÉSTO ESTÁ DEFINIDO EN ÉSTA FUNCIÓN move()...
        valid_moves = board.valid_moves()                                       #VEMOS QUÉ POSICIONES SON VÁLIDAD EN LA TABLA (Ej.solo tengo 7 podiciones libres)

        # exploracion                                                           #...TENEMOS LA FASE DE EXPLORACIÓN
        if explore and np.random.uniform(0, 1) < self.prob_exp:                 #SACAREMOS UN VALOR ALEATORIO ENTRE (0,1)->SI ES MÁS PEQUEÑO QUE LA
                                                                                #probabilidad de exploarción..
            # vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))                             #..ELEGIMOS UNA ACCIÓN TOTALMENTE ALEATORIA ENTRE LAS VÁLIDAS (valid_moves)
            return valid_moves[ix]                                              #DEVUELVE ESE VALOR.
                                                                                #FASE EXPLORATORIA: EL AGENTE EXPLORA DE FORMA ALEATORIA ACCIONES, DE TAL
                                                                                #FORMA QUE PUEDE SER QUE ENCUENTRE ALGO MEJOR DE TODO LO QUE HA ENCONTRADO
                                                                                #HASTA EL MOMENTO.

        # explotacion                                                           #FASE EXPLOTACIÓN: LAS ACCIONES QUE YO YA SÉ QUE SON BUENAS Y MALAS, VOY
                                                                                #A ELEGIR LA MEJOR QUE HAYA ENCONTRADO.
        # vamos a la posición con más valor
        max_value = -1000
        for row, col in valid_moves:                                            #PARA ELLO COGEMOS TODOS LOS MOVIMIENTOS VÁLIDOS
            next_board = board.state.copy()                                     #NOS CREAMOS UNA COPIA DE LA TABLA
            next_board[row, col] = self.symbol                                  #Y HACEMOS ESE MOVIMIENTO, VAMOS HACER TODOS ESOS MOVIMIENTOS VÁLIDOS  aquí le defines si es 1 o -1
            next_state = str(next_board.reshape(4*4))                           #COGER LA TABLA Y CONVERTIRLA EN UN string (cadena) [1 0 -1...]
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)   #Y PARA CADA UNO DE ÉSTOS MOVIMIENTOS VAMOS A
                                                                                #NUESTRA TABLA Y SACAR EL VALOR. SI NO LO ENCUENTRA EN LA TABLA ES UN ESTADO AL
                                                                                #QUE NUESTRO AGENTE NUNCA HA LLEGADO, POR TANTO LE PONGO UN VALOR DE CERO: value = 0
                                                                                #else self.value_function.get(next_state)->SI LO HA ENCONTRADO YA HA VISTO ESE ESTADO
                                                                                #EN ALGÚN MOMENTO Y POR TANTO YA TIENE UN VALOR->COGEMOS ESE VALOR
                                                                                #ÉSTO LO HAREMOS POR TODOS LOS MOVIMIENTOS VÁLIDOS..

            if value >= max_value:                                              #..QUEDÁNDONOS CON AQUEL MOVIMIENTO QUE NOS DÉ UN VALOR MÁS GRANDE DE FORMA QUE CUANDO EL AGENTE
                                                                                #HAYA PROBADO TODOS LOS MOVIMIENTOS QUE PUEDA HACER, SABE CUÁL ES, EL QUE EN PROMEDIO
                                                                                #LE VA A LLEVAR SIEMPRE A GANAR CON MAYOR PROBABILIDAD -> ESE ES EL QUE ELIGE.
                                                                                #EXPLOTAR EL CONOCIMIENTO QUE TIENE PARA HACER EL MEJOR MOVIMIENTO.
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col

    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))                    #está almacenando estados


    def reward(self, reward):                                                   #PERO CÓMO APRENDE: AL FINAL DE LA PARTIDA SE EMITEN ÉSTAS RECOMPENSAS
        # al final de la partida (cuando recibimos la recompensa)               #CADA AGENTE LLAMARÁ A ÉSTA FUNCIÓN reward(), AQUÍ RECIBIRÁ UN 0.5 SI HA
        # iteramos por tods los estados actualizando su valor en la tabla       #EMPATADO, 1 SI HA GANADO, 0 SI HA PERDIDO.
                                                                                #AQUÍ TENEMOS QUE ACTUALIZAR LA TABLA, IR POR TODAS LAS POSICIONES QUE HAYAMOS
                                                                                #VISTO DURANTE LA PARTIDA, Y EN LA TABLA TODAS AQUELLAS POR LAS QUE HAYA PASADO.
                                                                                #SI HA GANADO SUBIRLES EL VALOR Y SI HA PERDIDO BAJARLES EL VALOR.
                                                                                #DE FORMA QUE EN LA PRÓXIMA ITERACIÓN, SI POR LO QUE SEA LLEGA OTRA VEZ A ESE
                                                                                #ESTADO SABRÉ QUE YA GANÓ LA ANTERIOR VEZ Y HACE LA MISMA ACCIÓN. ó HACE 10 PARTIDAS
                                                                                #HE PERDIDO CON ÉSTA ACCIÓN, MEJOR OTRA ACCIÓN. SIGUE IMPLEMENTACIÓN.

        for p in reversed(self.positions):                                      #ITERAMOS POR TODAS LAS POSICIONES QUE HA PASADO DURANTE LA PARTIDA
                                                                                #->EN ÉSTE BUCLE ITERAMOS DE FORMA INVERTIDA, EMPEZAMOS POR LA ÚLTIMA POSICIÓN
                                                                                #PARA LA CUAL TENEMOS LA RECOMPENSA FINAL(1 ó 0)
            if self.value_function.get(p) is None:
                self.value_function[p] = 0                                      #SI ESAS POSICIONES NO ESTABAN EN LA TABLA, LAS INICIALIZAMOS A CERO
            self.value_function[p] += self.alpha * (reward - self.value_function[p])    #SI ESTABAN EN LA TABLA APLICAMOS ÉSTA FUNCIÓN..
                                                                                #COGEMOS NUESTRO VALOR: self.value_function[p] Y LE SUMAMOS  self.alpha (LEARNING
                                                                                #RATE) COMO EN LAS REDES NEURONALES -> ES UN HIPERPARÁMETRO QUE NOSOTROS CONTROLAMOS
                                                                                #Y CALCULAMOS LA DIFERENCIA ENTRE LA RECOMPENSA Y EL VALOR QUE TENEMOS:(reward - self.value_function[p])
                                                                                #SI LA RECOMPENSA ES 1, ÉSTA DIFERENCIA SERÁ POSITIVA Y NUESTRO VALOR AUMENTARÁ.
                                                                                #SI HEMOS PERDIDO ÉSTA RECOMPENSA SERÁ NEGATIVA Y EL VALOR DESCENDERÁ PARA ESE ESTADO(POSICIÓN).
            reward = self.value_function[p]     #Siempre y cuando esos estados existan en la tabla    #->PARA LA PENÚLTIMA ACCIÓN LA RECOMPENSA ES EL VALOR DEL ÚLTIMO ESTADO.
                                                                                #ASÍ DE FORMA ITERATIVA: DESDE LA ÚLTIMA ACCIÓN QUE TOMASTE HASTA LA PRIMERA
                                                                                #IREMOS ACTUALIZANDO ÉSTOS VALORES: HACIA ARRIBA SI HEMOS GANADO, HACIA ABAJO SI
                                                                                #HEMOS PERDIDO.

## ENTRENAMIENTO DE DOS AGENTES:
agent1 con una probabilidad de exploración de 0.5\
agent2 no le he puesto nada pero como prob_exp = 0.5 ya está definida, va a ser exactamente igual.\
-Voy hacer que jueguen 300000 partidas el uno contra el otro:\
-Son 28:02 minutos que juegan (no está optimizado, podría ser más rápido).\
-Recomendación: probar diferentes prob-exp, learning rates, diferentes números de partidas y a evaluar que es lo que se consigue.



In [ ]:
agent1 = Agent(prob_exp=0.5)
agent2 = Agent()

game = Game(agent1, agent2)

game.selfplay(100000)

100%|██████████| 100000/100000 [31:31<00:00, 52.88it/s]


[35321, 27124]

en éste caso el agent1 gana 34184 partidas, el agent2 28171.\
luego: sacamos la función de valor del agent1: agent1.value_function.items()\
lo convertimos en una tabla con pandas y podemos mostrar.




In [ ]:
import pandas as pd

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla

,estado,valor
0,[ 0. 0. 0. 1. 0. 0. -1. 1. 0. -1. 0. ...,1.0
1,[ 0. 0. 0. 1. -1. -1. -1. 1. 0. 0. 0. ...,1.0
2,[-1. -1. -1. 1. 0. 0. 0. 1. 0. 0. 0. ...,1.0
3,[ 0. 0. -1. 1. 0. 0. -1. 1. 0. 0. 0. ...,1.0
4,[ 0. 0. -1. 1. 0. 0. -1. 1. 0. 0. -1. ...,1.0
...,...,...
542316,[-1. 0. 0. 1. 1. 1. -1. 1. -1. 0. -1. -...,0.0
542317,[-1. 0. 0. 1. 1. 1. -1. 1. -1. 0. 0. -...,0.0
542318,[-1. 0. 0. 0. 1. 1. -1. 1. -1. 0. 0. -...,0.0
542319,[-1. 0. 0. 0. 1. 1. 0. 1. -1. 0. 0. -...,0.0


se ha visto 540036 estados diferentes durante todas las partidas.\
la representación del estado de la partida es: [0  0  0  1  0  0  0  1  .....1
 .....1]\
donde esta el jugador1 -> 1\
donde esta el jugador2 -> -1\
cero si no hay nadie\
y para cada uno de éstos estados tenemos un valor que va de 0 a 1.\
valor 1 es un estado en el que el jugador1 gana.\
valor 0 es un estado en el que el jugador1 pierde.\

-ahora con ésta tabla: en cada turno nuestro agente puede decir: cuáles son los posibles movimientos que puedo hacer -> para cada uno de ellos vendrá a ésta tabla y tomará el que tenga el valor más grande. sabe que si lleva a cabo esa acción, el estado siguiente al que va, en promedio le va hacer ganar más partidas de las que le va a hacer perder porque tienen un valor muy alto.


